# Module 5: Scale up your app

### Objectives

* Scale an app using kubectl.

### Scaling an application

In the previous modules we created a [Deployment](https://kubernetes.io/docs/concepts/workloads/controllers/deployment/), and then exposed it publicly via a [Service](https://kubernetes.io/docs/concepts/services-networking/service/). The Deployment created only one Pod for running our application. When traffic increases, we will need to scale the application to keep up with user demand.

**Scaling** is accomplished by changing the number of replicas in a Deployment

>You can create from the start a Deployment with multiple instances using the --replicas parameter for the kubectl create deployment command


## Scaling overview

<div><img src="https://d33wubrfki0l68.cloudfront.net/043eb67914e9474e30a303553d5a4c6c7301f378/0d8f6/docs/tutorials/kubernetes-basics/public/images/module_05_scaling1.svg" width="400">
<img src="https://d33wubrfki0l68.cloudfront.net/30f75140a581110443397192d70a4cdb37df7bfc/b5f56/docs/tutorials/kubernetes-basics/public/images/module_05_scaling2.svg" width="400">
</div>


Scaling out a Deployment will ensure new Pods are created and scheduled to Nodes with available resources. Scaling will increase the number of Pods to the new desired state. Kubernetes also supports [autoscaling](https://kubernetes.io/docs/user-guide/horizontal-pod-autoscaling/) of Pods, but it is outside of the scope of this tutorial. Scaling to zero is also possible, and it will terminate all Pods of the specified Deployment.

Running multiple instances of an application will require a way to distribute the traffic to all of them. Services have an integrated load-balancer that will distribute network traffic to all Pods of an exposed Deployment. Services will monitor continuously the running Pods using endpoints, to ensure the traffic is sent only to available Pods.

> Scaling is accomplished by changing the number of replicas in a Deployment.



Once you have multiple instances of an Application running, you would be able to do Rolling updates without downtime. We'll cover that in the next module. Now, let's go and scale our application.

### Lab 5 - Scaling up your app
#### Step 1 of 3: Scaling a deployment

To list your deployments use the get deployments command: kubectl get deployments

In [3]:
studentId=$(grep hpecp-user $HOME/.kube/config | cut -d= -f2)
POD_NAME=$(kubectl get pods -o go-template --template '{{range .items}}{{.metadata.name}}{{"\n"}}{{end}}' | grep $studentId) 
port=$(expr $(echo ${studentId} | tr -cd '[[:digit:]]') + 8000)

kubectl get deployments 


NAME                              READY   UP-TO-DATE   AVAILABLE   AGE
kubernetes-bootcamp-student1125   1/1     1            1           5d21h


The output should be similar to:

````
NAME                  READY   UP-TO-DATE   AVAILABLE   AGE
kubernetes-bootcamp   1/1     1            1           11m
````

- *NAME* lists the names of the Deployments in the cluster.
- *READY* shows the ratio of CURRENT/DESIRED replicas
- *UP-TO-DATE* displays the number of replicas that have been updated to achieve the desired state.
- *AVAILABLE* displays how many replicas of the application are available to your users.
- *AGE* displays the amount of time that the application has been running.

To see the ReplicaSet created by the Deployment, run kubectl get rs



In [4]:
kubectl get rs

NAME                                         DESIRED   CURRENT   READY   AGE
kubernetes-bootcamp-student1125-55cc476c66   1         1         1       5d21h


Notice that the name of the ReplicaSet is always formatted as [DEPLOYMENT-NAME]-[RANDOM-STRING]. The random string is randomly generated and uses the pod-template-hash as a seed.

Two important columns of this command are:

- *DESIRED* displays the desired number of replicas of the application, which you define when you create the Deployment. This is the desired state.
- *CURRENT* displays how many replicas are currently running.

Next, let’s scale the Deployment to 4 replicas. We’ll use the kubectl scale command, followed by the deployment type, name and desired number of instances:

In [5]:
kubectl scale deployments/kubernetes-bootcamp-$studentId --replicas=4

deployment.apps/kubernetes-bootcamp-student1125 scaled


To list your Deployments once again, use get deployments:

In [6]:
kubectl get deployments

NAME                              READY   UP-TO-DATE   AVAILABLE   AGE
kubernetes-bootcamp-student1125   4/4     4            4           5d21h


The change was applied, and we have 4 instances of the application available. Next, let’s check if the number of Pods changed:

In [9]:
kubectl get pods -o wide

NAME                                               READY   STATUS    RESTARTS   AGE     IP             NODE                               NOMINATED NODE   READINESS GATES
kubernetes-bootcamp-student1125-55cc476c66-gjdr4   1/1     Running   0          5d21h   10.192.0.144   worker27.etc.fr.comm.hpecorp.net   <none>           <none>
kubernetes-bootcamp-student1125-55cc476c66-nwvh2   1/1     Running   0          103s    10.192.1.11    worker28.etc.fr.comm.hpecorp.net   <none>           <none>
kubernetes-bootcamp-student1125-55cc476c66-t6hgq   1/1     Running   0          103s    10.192.0.149   worker27.etc.fr.comm.hpecorp.net   <none>           <none>
kubernetes-bootcamp-student1125-55cc476c66-zqjk5   1/1     Running   0          103s    10.192.1.12    worker28.etc.fr.comm.hpecorp.net   <none>           <none>


There are 4 Pods now, with different IP addresses. The change was registered in the Deployment events log. To check that, use the describe command:

In [10]:
kubectl describe deployments/kubernetes-bootcamp-$studentId

Name:                   kubernetes-bootcamp-student1125
Namespace:              k8shacktenant
CreationTimestamp:      Thu, 28 Jan 2021 12:45:22 +0000
Labels:                 app=kubernetes-bootcamp-student1125
Annotations:            deployment.kubernetes.io/revision: 1
Selector:               app=kubernetes-bootcamp-student1125
Replicas:               4 desired | 4 updated | 4 total | 4 available | 0 unavailable
StrategyType:           RollingUpdate
MinReadySeconds:        0
RollingUpdateStrategy:  25% max unavailable, 25% max surge
Pod Template:
  Labels:  app=kubernetes-bootcamp-student1125
  Containers:
   kubernetes-bootcamp:
    Image:        gcr.io/google-samples/kubernetes-bootcamp:v1
    Port:         <none>
    Host Port:    <none>
    Environment:  <none>
    Mounts:       <none>
  Volumes:        <none>
Conditions:
  Type           Status  Reason
  ----           ------  ------
  Progressing    True    NewReplicaSetAvailable
  Available      True    MinimumReplicasAvailable

You can also view in the output of this command that there are 4 replicas now.

#### Step 2 of 3: Load Balancing

Let’s check that the Service is load-balancing the traffic. To find out the exposed IP and Port we can use the describe service as we learned in the previous Module:

In [ ]:
kubectl expose deployment/kubernetes-bootcamp-$studentId --type="NodePort" --port 8080

In [15]:
kubectl describe service/kubernetes-bootcamp-$studentId  

Name:                     kubernetes-bootcamp-student1125
Namespace:                k8shacktenant
Labels:                   app=kubernetes-bootcamp-student1125
Annotations:              <none>
Selector:                 app=kubernetes-bootcamp-student1125
Type:                     NodePort
IP:                       10.107.67.230
Port:                     <unset>  8080/TCP
TargetPort:               8080/TCP
NodePort:                 <unset>  32574/TCP
Endpoints:                10.192.0.144:8080,10.192.0.149:8080,10.192.1.11:8080 + 1 more...
Session Affinity:         None
External Traffic Policy:  Cluster
Events:                   <none>


Create an environment variable called NODE_PORT that has the value of the Node port assigned.
And another one that compute NODE_NAME from a  `kubectl describe pods ` command. 

In [49]:
NODE_PORT=$(kubectl get services/kubernetes-bootcamp-$studentId -o go-template='{{(index .spec.ports 0).nodePort}}')
echo NODE_PORT=$NODE_PORT
NODE_NAME=$(kubectl describe pods $POD_NAME | grep Node:)
NODE_NAME=${NODE_NAME#Node:} 
NODE_NAME=${NODE_NAME%/*} 
NODE_NAME=$(echo $NODE_NAME | xargs)
echo NODE_NAME=$NODE_NAME


NODE_PORT=32574
NODE_NAME=worker27.etc.fr.comm.hpecorp.net


Next, we’ll do a `curl` to the exposed IP and port. Execute the command multiple times:

> To execute a multiple times, use CTRL-Enter while positioned on that cell.

In [59]:
curl http://$NODE_NAME:$NODE_PORT

Hello Kubernetes bootcamp! | Running on: kubernetes-bootcamp-student1125-55cc476c66-gjdr4 | v=1


We hit a different Pod with every request. This demonstrates that the load-balancing is working.

#### Step 3 of 3: Deleting a service

To delete Services you can use the `delete service` command. Labels can be used also here:

In [60]:
kubectl delete service -l app=kubernetes-bootcamp-$studentId

service "kubernetes-bootcamp-student1125" deleted


Confirm that the service is gone:

In [61]:
kubectl get services | grep $studentId

No resources found in k8shacktenant namespace.


: 1

This confirms that our Service was removed. To confirm that route is not exposed anymore you can curl the previously exposed IP and port:

In [62]:
curl http://$NODE_NAME:$NODE_PORT

curl: (7) Failed to connect to worker27.etc.fr.comm.hpecorp.net port 32574: Connection refused


: 7

This proves that the app is not reachable anymore from outside of the cluster. You can confirm that the app is still running with a curl inside the pod:

In [63]:
kubectl exec $POD_NAME -- curl -s localhost:8080

Hello Kubernetes bootcamp! | Running on: kubernetes-bootcamp-student1125-55cc476c66-gjdr4 | v=1


We see here that the application is up. This is because the Deployment is managing the application. To shut down the application, you would need to delete the Deployment as well.


This terminates module 5. Let's continue with [module 6, Update your app](./7-WKSHP-K8S101-mod6.ipynb).